In [4]:
%pip install ibm_cloud_sdk_core --no-cache
%pip install --upgrade ibm-watson-openscale
%pip install ibm-cos-sdk --no-cache
%pip install botocore==1.23.19 --no-cache

In [5]:
import pandas as pd
import numpy as np

In [9]:
cloud_WOS = True
API_KEY = 'API_KEY'

DB_CREDENTIALS=None

SCHEMA_NAME = 'azure_dm'

In [10]:
IAM_URL="https://iam.ng.bluemix.net/oidc/token"

In [11]:
COS_API_KEY_ID = "***"
COS_RESOURCE_CRN = "***"
COS_ENDPOINT = "***" 
BUCKET_NAME = "***"

In [16]:
AZURE_ENGINE_CREDENTIALS =  {
    "client_id": 'CLIENT_ID',
    "client_secret": 'CLIENT_SECRET',
    "tenant": 'TENANT',
    "subscription_id": 'SUBSCRIPTION_ID'
}

NOTE: read from `/data` directory if running locally

In [13]:
df = pd.read_csv('../data/credit_risk_training.csv')
df.head()

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
1,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,...,savings_insurance,37,stores,own,2,skilled,1,none,yes,No Risk
2,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,...,real_estate,28,none,own,2,skilled,1,yes,no,No Risk
3,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
4,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,...,unknown,57,none,own,2,skilled,1,none,yes,Risk


In [15]:
print('Columns: ', list(df.columns))
print('Number of columns: ', len(df.columns))

Columns:  ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'Risk']
Number of columns:  21


In [17]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [19]:

train_data, test_data = train_test_split(df, test_size=0.2)

In [20]:
features_idx = np.s_[0:-1]
all_records_idx = np.s_[:]
first_record_idx = np.s_[0]

In [26]:
string_fields = [type(fld) is str for fld in train_data.iloc[first_record_idx, features_idx]]
ct = ColumnTransformer([("ohe", OneHotEncoder(), list(np.array(train_data.columns)[features_idx][string_fields]))])
clf_linear = SGDClassifier(loss='log_loss', penalty='l2', max_iter=1000, tol=1e-5)

pipeline = Pipeline([('ct', ct), ('clf_linear', clf_linear)])

In [24]:
MODEL_NAME = "Scikit Credit Risk Model Azure ML"
MODEL_NAME_SHORT = 'azure_credit_risk_model'
DEPLOYMENT_NAME = "Scikit Credit Risk Deployment Azure ML"

In [27]:
risk_model = pipeline.fit(train_data.drop('Risk', axis=1), train_data.Risk)

In [28]:
from sklearn.metrics import roc_auc_score

predictions = risk_model.predict(test_data.drop('Risk', axis=1))
indexed_preds = [0 if prediction=='No Risk' else 1 for prediction in predictions]

real_observations = test_data.Risk.replace('Risk', 1)
real_observations = real_observations.replace('No Risk', 0).values

auc = roc_auc_score(real_observations, indexed_preds)
print(auc)

0.7396359427609427


In [29]:
import joblib

joblib.dump(risk_model, MODEL_NAME_SHORT+".pkl")

['azure_credit_risk_model.pkl']

In [36]:
%pip install azureml.core --no-cache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 146.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 116.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.9/349.9 kB 7.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.6/656.6 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [38]:
import azureml.core

print('Azure ML SDK version:', azureml.core.VERSION)

Azure ML SDK version: 1.54.0


In [40]:
from azureml.core import Workspace


ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

workspace_name = ws.name

UserErrorException: UserErrorException:
	Message: The workspace configuration file config.json, could not be found in /Users/leonardofurnielis/Documents/github/wml-toolkit/jupyter or its parent directories. Please check whether the workspace configuration file exists, or provide the full path to the configuration file as an argument. You can download a configuration file for your workspace, via http://ml.azure.com and clicking on the name of your workspace in the right top.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "The workspace configuration file config.json, could not be found in /Users/leonardofurnielis/Documents/github/wml-toolkit/jupyter or its parent directories. Please check whether the workspace configuration file exists, or provide the full path to the configuration file as an argument. You can download a configuration file for your workspace, via http://ml.azure.com and clicking on the name of your workspace in the 